limpieza y preparación de datos de freelancers globales
en este cuaderno, se realizará un proceso de limpieza y preparación de datos (data cleaning) sobre el dataset global_freelancers_raw.csv. el objetivo es identificar y corregir problemáticas comunes como valores nulos, inconsistencias en los datos y formatos incorrectos para dejar el dataset listo para un análisis más profundo.

1. carga de datos y exploración inicial
En primer lugar, se importan las librerías necesarias y se carga el archivo CSV en un DataFrame de pandas.
Posteriormente, se realiza una revisión inicial de su estructura mediante las funciones sample() y info(), lo que permite identificar los tipos de datos, la presencia de valores nulos y posibles inconsistencias.

Los resultados iniciales evidencian:

Presencia de valores faltantes en variables como age, years_of_experience, hourly_rate (USD), rating, is_active y client_satisfaction.

Variables numéricas almacenadas como texto (object), como hourly_rate (USD) y client_satisfaction.

Inconsistencias en columnas categóricas como gender e is_active.

In [2]:
pip install scikit-learn

   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ------- -------------------------------- 1.6/8.7 MB 9.1 MB/s eta 0:00:01
   ---------------- ----------------------- 3.7/8.7 MB 9.3 MB/s eta 0:00:01
   ------------------------- -------------- 5.5/8.7 MB 9.3 MB/s eta 0:00:01
   -------------------------------- ------- 7.1/8.7 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 8.6 MB/s  0:00:01
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   - -------------------------------------- 1.8/38.5 MB 9.2 MB/s eta 0:00:04
   --- ------------------------------------ 3.7/38.5 MB 9.6 MB/s eta 0:00:04
   ----- ---------------------------------- 5.5/38.5 MB 9.4 MB/s eta 0:00:04
   ------- -------------------------------- 7.1/38.5 MB 9.0 MB/s eta 0:00:04
   --------- ------------------------------ 9.2/38.5 MB 9.0 MB/s eta 0:00:04
   ----------- ---------------------------- 11.0/38.5 MB 9.0 MB/s eta 0:00:04
   ------------- ----

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

# Cargar el dataset desde la ruta proporcionada
# Asegúrate de que el archivo 'global_freelancers_raw.csv' esté en esta ruta
file_path = r'C:\Users\user\Downloads\global_freelancers_raw.csv'
df = pd.read_csv(file_path)

# Mostrar una muestra aleatoria de los datos y la información general
print("Muestra de los datos:")
print(df.sample(5))
print("\nInformación general del DataFrame:")
df.info()


Muestra de los datos:
    freelancer_ID                name  gender   age  country    language  \
822      FL250823         Erin Martin  female  37.0    Japan    Japanese   
819      FL250820      Alexandra Sims  Female  40.0    India       Hindi   
762      FL250763  Christopher Sparks       m  37.0  Germany      German   
67       FL250068         Vicki Kelly  FEMALE  55.0   Brazil  Portuguese   
262      FL250263        Jacob Joseph    Male  48.0    Italy     Italian   

              primary_skill  years_of_experience hourly_rate (USD)  rating  \
822                      AI                  6.0                20     NaN   
819  Blockchain Development                 14.0                75     0.0   
762          Graphic Design                 12.0               NaN     4.9   
67          Web Development                  0.0               $40     0.0   
262         Web Development                 23.0               NaN     2.1   

    is_active client_satisfaction  
822       NaN   

A partir del diagnóstico anterior, se abordan las principales problemáticas detectadas:

Problemática 1: Inconsistencias en la columna gender

La variable gender presentaba múltiples representaciones del mismo valor (por ejemplo, ‘f’, ‘female’, ‘FEMALE’).
Se estandarizó la columna, normalizando los valores a ‘Female’ y ‘Male’, garantizando coherencia en el formato.



In [4]:
# valores únicos antes de la limpieza
print("valores únicos en 'gender' antes de la limpieza:")
print(df['gender'].unique())

# estandarizar la columna 'gender' a 'female' y 'male'
# se utiliza una función lambda que evalúa si el texto comienza con 'f' (ignorando mayúsculas/minúsculas)
df['gender'] = df['gender'].str.strip().str.lower().apply(lambda x: 'female' if str(x).startswith('f') else 'male')

# verificar los cambios
print("\nvalores únicos en 'gender' después de la limpieza:")
print(df['gender'].unique())

valores únicos en 'gender' antes de la limpieza:
['f' 'FEMALE' 'male' 'F' 'female' 'm' 'MALE' 'Female' 'M' 'Male']

valores únicos en 'gender' después de la limpieza:
['female' 'male']


Problemática 2: Formato Incorrecto en hourly_rate (USD)

La columna de tarifa por hora contenía símbolos monetarios y códigos de divisa ($, USD), lo que impedía su tratamiento numérico.
Se limpiaron los caracteres no numéricos y se transformó la variable al tipo float, permitiendo realizar cálculos y estadísticas correctamente.


In [9]:
# forzar la conversión de toda la columna a tipo string
df['hourly_rate (USD)'] = df['hourly_rate (USD)'].astype(str)

# ahora sí, eliminar caracteres no numéricos
df['hourly_rate (USD)'] = df['hourly_rate (USD)'].str.replace('USD', '', regex=False)
df['hourly_rate (USD)'] = df['hourly_rate (USD)'].str.replace('$', '', regex=False)

# convertir la columna a tipo numérico. los errores se convertirán en nan.
df['hourly_rate (USD)'] = pd.to_numeric(df['hourly_rate (USD)'], errors='coerce')



Problemática 3: Formato Inconsistente en client_satisfaction

La variable de satisfacción incluía el símbolo ‘%’ y estaba almacenada como texto.
Se eliminaron los símbolos y se convirtió a tipo numérico, dejando los valores expresados como porcentajes enteros.

In [10]:
# eliminar el símbolo de porcentaje
df['client_satisfaction'] = df['client_satisfaction'].str.replace('%', '', regex=False)

# convertir la columna a tipo numérico, manejando errores.
df['client_satisfaction'] = pd.to_numeric(df['client_satisfaction'], errors='coerce')

print("tipo de dato de 'client_satisfaction' después de la conversión:", df['client_satisfaction'].dtype)

tipo de dato de 'client_satisfaction' después de la conversión: float64


Problemática 4: Estandarización de la columna is_active

La columna presentaba distintas formas de representar un valor booleano (por ejemplo, ‘True’, ‘1’, ‘Y’, ‘yes’, ‘N’, ‘0’, ‘False’).
Se unificó la codificación estableciendo los valores ‘Yes’ y ‘No’, eliminando así la ambigüedad en los datos categóricos.

In [11]:
# valores únicos antes de la limpieza
print("valores únicos en 'is_active' antes de la limpieza:")
print(df['is_active'].unique())

# estandarizar la columna a valores 'yes' y 'no'
# la función convierte todo a string, limpia espacios, pasa a minúsculas y verifica si comienza con 't', 'y', o '1'
df['is_active'] = df['is_active'].apply(
    lambda x: 'yes' if str(x).strip().lower().startswith(('t', 'y', '1')) else 'no'
)

# verificar los cambios
print("\nvalores únicos en 'is_active' después de la limpieza:")
print(df['is_active'].unique())

valores únicos en 'is_active' antes de la limpieza:
['0' '1' 'N' 'False' 'True' 'yes' 'Y' nan 'no']

valores únicos en 'is_active' después de la limpieza:
['no' 'yes']


Problemática 5: Imputación de Valores Nulos

Para abordar los valores faltantes:

Se imputó la variable age mediante su media, dada la baja proporción de nulos.

Se aplicó el método KNNImputer a las columnas years_of_experience, hourly_rate (USD), rating y client_satisfaction, aprovechando las relaciones entre variables para estimar los valores ausentes con mayor precisión.
Finalmente, se redondearon los resultados para mejorar la presentación del conjunto de datos.

In [12]:
# calcular el porcentaje de valores nulos
print("porcentaje de nulos antes de la imputación:")
print(df.isnull().sum() / len(df) * 100)

# 1. imputación con la media para 'age'
df['age'].fillna(df['age'].mean(), inplace=True)

# 2. imputación con knn para las columnas restantes
# seleccionamos las columnas numéricas que necesitan imputación
columns_to_impute = ['years_of_experience', 'hourly_rate (USD)', 'rating', 'client_satisfaction']

# inicializamos el imputador knn
knn_imputer = KNNImputer(n_neighbors=5)

# aplicamos la transformación
df[columns_to_impute] = knn_imputer.fit_transform(df[columns_to_impute])

# redondear los valores para que tengan un formato más limpio
df['years_of_experience'] = df['years_of_experience'].round(0)
df['rating'] = df['rating'].round(1)
df['client_satisfaction'] = df['client_satisfaction'].round(0)


print("\nverificación de valores nulos después de la imputación:")
print(df.isnull().sum())

porcentaje de nulos antes de la imputación:
freelancer_ID           0.0
name                    0.0
gender                  0.0
age                     3.0
country                 0.0
language                0.0
primary_skill           0.0
years_of_experience     5.1
hourly_rate (USD)       9.4
rating                 10.1
is_active               0.0
client_satisfaction    17.6
dtype: float64

verificación de valores nulos después de la imputación:
freelancer_ID          0
name                   0
gender                 0
age                    0
country                0
language               0
primary_skill          0
years_of_experience    0
hourly_rate (USD)      0
rating                 0
is_active              0
client_satisfaction    0
dtype: int64


C:\Users\user\AppData\Local\Temp\ipykernel_19856\1071419881.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(df['age'].mean(), inplace=True)


después de aplicar todos los pasos de limpieza, el dataset está estandarizado y libre de valores nulos, listo para ser utilizado en análisis exploratorio, visualización de datos y modelado.

In [13]:
# mostrar una muestra del dataframe limpio
print("\nmuestra del dataframe limpio:")
print(df.head())

# mostrar la información final del dataframe
print("\ninformación final del dataframe limpio:")
df.info()


muestra del dataframe limpio:
  freelancer_ID             name  gender   age    country language  \
0      FL250001  Ms. Nicole Kidd       f  52.0      Italy  Italian   
1      FL250002   Vanessa Garcia  FEMALE  52.0  Australia  English   
2      FL250003      Juan Nelson    male  53.0    Germany   German   
3      FL250004   Amanda Spencer       F  38.0  Australia  English   
4      FL250005  Lynn Curtis DDS  female  53.0    Germany   German   

            primary_skill  years_of_experience  hourly_rate (USD)  rating  \
0  Blockchain Development                 11.0              100.0     2.4   
1             Mobile Apps                 34.0              100.0     3.3   
2          Graphic Design                 31.0               50.0     0.0   
3         Web Development                  4.0               40.0     1.5   
4         Web Development                 27.0               30.0     4.8   

  is_active  client_satisfaction  
0        no                 77.0  
1       yes    

Conclusión:
El proceso de limpieza y preparación de datos aplicado al dataset global_freelancers_raw.csv permitió transformar un conjunto de datos desordenado y con inconsistencias en una base estructurada, coherente y lista para el análisis.
A través de la estandarización de valores categóricos, la corrección de formatos numéricos y la imputación de valores faltantes mediante técnicas como KNNImputer, se logró mejorar la calidad y confiabilidad de la información.

Como resultado, el dataset ahora presenta:

Variables consistentes y correctamente tipificadas.

Ausencia de valores nulos.

Datos listos para su análisis estadístico, visualización y modelado predictivo.

En conclusión, esta limpieza no solo optimiza la precisión de futuros análisis, sino que también garantiza una base sólida para la toma de decisiones basada en datos confiables y comparables.